In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pylab as plt
import datetime
import locale
locale.setlocale(locale.LC_NUMERIC, "")
import subprocess
from stock_util.data_processor import DataLoader
from stock_util.model import Model
import math


from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.recurrent import LSTM, GRU
from keras.layers import Conv1D, MaxPooling1D, AtrousConvolution1D, RepeatVector
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from keras.layers.wrappers import Bidirectional
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import *
from keras.optimizers import RMSprop, Adam, SGD, Nadam
from keras.initializers import *
from keras.backend import clear_session

Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
BID_QUERY='index="forex" sourcetype="stock:quote:eur_usd" NOT date_wday="saturday" NOT date_wday="sunday" earliest="4/15/2019:00:00:00" latest="4/20/2019:00:00:00"|where date_hour>=0 AND date_hour<17  | eval _time=strftime(_time,"%m-%d-%Y-%H:%M:%S") | eval bid.v=volume | eval ask.v=volume| table _time bid.o,bid.h,bid.l,bid.c,bid.v,date_wday,date_hour,date_minute,date_second | sort 0 _time'
ASK_QUERY='index="forex" sourcetype="stock:quote:eur_usd" NOT date_wday="saturday" NOT date_wday="sunday" earliest="4/15/2019:00:00:00" latest="4/20/2019:00:00:00"| where date_hour>=0 AND date_hour<17  |eval _time=strftime(_time,"%m-%d-%Y-%H:%M:%S") | eval bid.v=volume | eval ask.v=volume| table _time ask.o,ask.h,ask.l,ask.c,ask.v,date_wday,date_hour,date_minute,date_second | sort 0 _time'

In [3]:
loader=DataLoader()

In [4]:
def get_data():
    global loader
    loader.set_queries(BID_QUERY,ASK_QUERY)
    return loader.get_days(50000)

In [5]:
df=get_data()
print(df.head())
print(df.tail())

              Datetime   Open_x   High_x    Low_x  Close_x  Volume_x  \
0  04-15-2019-00:00:00  1.13115  1.13118  1.13111  1.13114    406.43   
1  04-15-2019-00:01:00  1.13113  1.13117  1.13113  1.13115    368.70   
2  04-15-2019-00:02:00  1.13114  1.13124  1.13114  1.13124    455.28   
3  04-15-2019-00:03:00  1.13123  1.13129  1.13123  1.13123    441.73   
4  04-15-2019-00:04:00  1.13123  1.13128  1.13123  1.13127    235.54   

  Weekday_x   Open_y   High_y    Low_y  Close_y  Volume_y Weekday_y  
0    monday  1.13111  1.13115  1.13110  1.13112    406.43    monday  
1    monday  1.13112  1.13122  1.13112  1.13121    368.70    monday  
2    monday  1.13121  1.13127  1.13120  1.13122    455.28    monday  
3    monday  1.13121  1.13126  1.13121  1.13125    441.73    monday  
4    monday  1.13126  1.13127  1.13122  1.13122    235.54    monday  
                 Datetime   Open_x   High_x    Low_x  Close_x  Volume_x  \
5090  04-19-2019-16:54:00  1.12450  1.12470  1.12441  1.12441     99.59 

In [6]:
def printvalue(array,i):
    print(array[i][0])

In [44]:
WINDOW=60
FORECAST=90

In [45]:
X_train, X_test, y_train, y_test = loader.get_train_test(df,0.001)
loader.window=WINDOW
loader.forecast=FORECAST
model=Model(loader.window,10,"test.h5","forex.h5",loader.forecast,50)
model.set_logging(0)
model.build_model()
pred=model.predict(X_test)

Doing ternary tensor with time length of  5095  and  1 steps
0 windows sized
500 windows sized
1000 windows sized
1500 windows sized
2000 windows sized
2500 windows sized
3000 windows sized
3500 windows sized
4000 windows sized
Tensor X/Y len:  4345  /  4345

Testing dataset dimension: [ 4341 / 4 ]



In [91]:
X_test.shape
frames=X_test[:]
print(frames[:,3,3].max())
print(frames[:,3,3].min())
print(X_test[0,12,3])

3.407537339804756
-3.431977669091021
0.30354108217319203


In [94]:

    
def zlimit_test(pred,X_test,y_test):
    
    actions = []
    dev = []
    right = 0
    wrong = 0
    
    df_closes=df.Close_x.tolist()
    dfs=X_test[:]
    z_closes=dfs[:,3,3]
    
    for i in range(len(pred)):
        #volatility[i]=math.sqrt(df.loc[i:loader.window-1].var())
        
        d=np.std(df_closes[i:i+loader.window],dtype=np.float32)
        dev.append(d)
        
    #for limit in [49]:
    minz=round(np.min(z_closes)*10,0)
    maxz=round(np.max(z_closes)*10,0)
    print ("Using minz/maxz: ",str(minz),"/",str(maxz))

    for zlimit in range(int(minz),int(maxz)):
        actions=[]
        right=0
        wrong=0
        limit=float(round(49/100,2))
        zlimit=round(zlimit,3)
        vol_skipped=0
        z_skipped=0
        
        for x in range(len(pred)):
            traded=0
            right_wrong=""
            zscore=X_test[x][WINDOW-1][3].astype(float)
            #print("Zscore:",str(zscore))
            zscore=int(zscore*10)
            
            if zscore > zlimit:
                z_skipped=z_skipped+1
                continue
                
            #if dev[x] < 0.0001:
            #    vol_skipped=vol_skipped+1
                #print("Skipping Vol:", volatility[x])
            #    continue
                
            if float(pred[x][0]-pred[x][2])>limit:
                traded=1
                if y_test[x][0]==1:
                    right=right+1
                    right_wrong="right"
                else:
                    wrong=wrong+1
                    right_wrong="wrong"
            elif float(pred[x][1]-pred[x][2])>limit:
                traded=1
                if y_test[x][1]==1:
                    right=right+1
                    right_wrong="right"
                else:
                    wrong=wrong+1
                    right_wrong="wrong"
            
            #if traded > 0:
            #    print("Right: ",str(right),"Wrong: ",str(wrong)," Volatility: ",str(volatility[x]), " ",right_wrong)
                    
        if right > 0:
            print ("ZLIMIT: ",str(zlimit)," RIGHT: "+str(right)+" | WRONG: ", str(wrong), " | SKIP: ",str(vol_skipped), " | ZSKIP: ",str(z_skipped)," | RIGHT PERCENTAGE:", str(round((right*100)/(right+wrong),1)),"%")

    

In [141]:
def stddev_test(pred,X_test,y_test):
    
    actions = []
    dev = []
    right = 0
    wrong = 0
    
    df_closes=df.Close_x.tolist()
    
    for i in range(len(pred)):
        #volatility[i]=math.sqrt(df.loc[i:loader.window-1].var())
        
        d=np.std(df_closes[i:i+WINDOW-1],dtype=np.float32)
        dev.append(d)
    
    for limit in [49]:
        actions=[]
        right=0
        wrong=0
        limit=float(round(limit/100,2))
        skipped=0
        print("\n")
        averages=[]
        
        for limit2 in np.arange(0.00004,0.0003,0.000010):
            right=0
            wrong=0
            skipped=0
            
            for x in range(len(pred)):
                traded=0
                right_wrong=""

                if dev[x] < limit2:
                    skipped=skipped+1
                    #print("Skipping Vol:", volatility[x])
                    continue

                if float(pred[x][0]-pred[x][2])>limit:
                    traded=1
                    if y_test[x][0]==1:
                        right=right+1
                        right_wrong="right"
                    else:
                        wrong=wrong+1
                        right_wrong="wrong"
                elif float(pred[x][1]-pred[x][2])>limit:
                    traded=1
                    if y_test[x][1]==1:
                        right=right+1
                        right_wrong="right"
                    else:
                        wrong=wrong+1
                        right_wrong="wrong"

                #if traded > 0:
                #    print("Right: ",str(right),"Wrong: ",str(wrong)," Volatility: ",str(volatility[x]), " ",right_wrong)

            if right > 0:
                avg=round((right*100)/(right+wrong),1)
                averages.append(avg)
                print ("LIMIT: ",str(limit), "LIMIT2: ",str(limit2)," | RIGHT: "+str(right)+" | WRONG: ", str(wrong), " | SKIP: ",str(skipped), " | RIGHT PERCENTAGE:", str(round((right*100)/(right+wrong),1)),"%")
        print("Average/Min/Max For LIMIT ",str(limit)," : ",str(round(np.mean(averages),1)),"/",str(round(np.min(averages),1)),"/",str(round(np.max(averages),1)))

In [142]:
#zlimit_test(pred,X_test,y_test)
stddev_test(pred,X_test,y_test)



LIMIT:  0.49 LIMIT2:  4e-05  | RIGHT: 375 | WRONG:  303  | SKIP:  18  | RIGHT PERCENTAGE: 55.3 %
LIMIT:  0.49 LIMIT2:  5e-05  | RIGHT: 374 | WRONG:  303  | SKIP:  51  | RIGHT PERCENTAGE: 55.2 %
LIMIT:  0.49 LIMIT2:  6e-05  | RIGHT: 365 | WRONG:  286  | SKIP:  146  | RIGHT PERCENTAGE: 56.1 %
LIMIT:  0.49 LIMIT2:  7e-05  | RIGHT: 359 | WRONG:  265  | SKIP:  273  | RIGHT PERCENTAGE: 57.5 %
LIMIT:  0.49 LIMIT2:  7.999999999999999e-05  | RIGHT: 357 | WRONG:  254  | SKIP:  353  | RIGHT PERCENTAGE: 58.4 %
LIMIT:  0.49 LIMIT2:  8.999999999999999e-05  | RIGHT: 347 | WRONG:  239  | SKIP:  534  | RIGHT PERCENTAGE: 59.2 %
LIMIT:  0.49 LIMIT2:  9.999999999999999e-05  | RIGHT: 314 | WRONG:  226  | SKIP:  765  | RIGHT PERCENTAGE: 58.1 %
LIMIT:  0.49 LIMIT2:  0.00010999999999999999  | RIGHT: 309 | WRONG:  212  | SKIP:  899  | RIGHT PERCENTAGE: 59.3 %
LIMIT:  0.49 LIMIT2:  0.00011999999999999999  | RIGHT: 306 | WRONG:  203  | SKIP:  1010  | RIGHT PERCENTAGE: 60.1 %
LIMIT:  0.49 LIMIT2:  0.00013  | RI